<a href="https://colab.research.google.com/github/JamorMoussa/2IA/blob/main/utils/mlp_ga_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch pygad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 1.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pygad
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
class MyDataset(Dataset):
    def __init__(self, size):
        self.X = torch.rand(size, 2)
        self.y = torch.matmul(self.X, torch.Tensor([1, 2]).t())

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [4]:
class IsNeuronAct(nn.Module):

  def __init__(self, mask: torch.Tensor):
      super(IsNeuronAct, self).__init__()
      self.mask = nn.Parameter(torch.Tensor(mask), requires_grad=False)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return x * self.mask

In [14]:
class MLP(nn.Module):

  def __init__(self, mask: torch.Tensor):
    super(MLP, self).__init__()

    self.l1 = nn.Linear(2, 4)
    self.act1 = IsNeuronAct(mask)
    self.re = nn.ReLU()
    self.l2 = nn.Linear(4, 1)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = self.l1(x)
    #x =self.re(x)
    x = self.act1(x)
    return self.l2(x)

In [15]:
dataset = MyDataset(size=1000)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [43]:
results: list[tuple[nn.Module, float]] = []

def train_model(mask, num_epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = MLP(mask)
    model.to(device)

    mse = nn.MSELoss()
    opt = torch.optim.SGD(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)

            loss = mse(outputs, targets.view(-1, 1))

            opt.zero_grad()
            loss.backward()
            opt.step()

    print(f"loss: {loss.item()}")
    results.append((model, loss.item()))
    return loss

In [17]:
train_model(torch.Tensor([1, 1, 1, 1]))

loss: 0.05276922881603241


tensor(0.0528, device='cuda:0', grad_fn=<MseLossBackward0>)

In [44]:
def fitness(ga_instance, solution, solution_idx):
    loss = train_model(solution)

    return 1/(float(loss)+ 1e-17) + 4 - float(np.sum(solution))

In [49]:
num_genes = 4
num_generations = 15
population_size = 5

initial_population = np.random.choice([0, 1], size=(population_size, num_genes))

In [50]:
ga_instance = pygad.GA(
    num_generations=num_generations,
    num_parents_mating=2,
    initial_population=initial_population.copy(),
    fitness_func=fitness,
    mutation_type="random",
    crossover_probability=0.8,
    mutation_probability=0.2,
    parent_selection_type="rank",
    crossover_type="single_point",
    keep_parents=2,
    gene_type=int,
)

In [51]:
ga_instance.run()

loss: 0.10145790129899979
loss: 0.1241721510887146
loss: 0.01204755064100027
loss: 0.2637237310409546
loss: 0.007575054652988911
loss: 0.15183675289154053
loss: 0.13855338096618652
loss: 0.03598170727491379
loss: 0.07619421929121017
loss: 0.050130151212215424
loss: 0.049641307443380356
loss: 0.06808394938707352
loss: 0.1720898449420929
loss: 0.20392201840877533
loss: 0.05708683282136917
loss: 0.1610950231552124
loss: 0.16839219629764557
loss: 0.45518702268600464
loss: 0.2026398777961731
loss: 0.2145734429359436


In [52]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()

print("Best solution:", solution)
print("Fitness value:", solution_fitness)

loss: 0.3882814943790436
Best solution: [1 0 1 1]
Fitness value: 133.01224886284163
